In [3]:
!conda init
!conda activate uqdd-118

no change     /home/bkhalil/miniconda3/condabin/conda
no change     /home/bkhalil/miniconda3/bin/conda
no change     /home/bkhalil/miniconda3/bin/conda-env
no change     /home/bkhalil/miniconda3/bin/activate
no change     /home/bkhalil/miniconda3/bin/deactivate
no change     /home/bkhalil/miniconda3/etc/profile.d/conda.sh
no change     /home/bkhalil/miniconda3/etc/fish/conf.d/conda.fish
no change     /home/bkhalil/miniconda3/shell/condabin/Conda.psm1
no change     /home/bkhalil/miniconda3/shell/condabin/conda-hook.ps1
no change     /home/bkhalil/miniconda3/lib/python3.11/site-packages/xontrib/conda.xsh
no change     /home/bkhalil/miniconda3/etc/profile.d/conda.csh
no change     /home/bkhalil/.bashrc
No action taken.

CondaError: Run 'conda init' before 'conda activate'



In [10]:
import os
import pandas as pd
import numpy as np
import torch
import glob
from uqdd import DEVICE, FIGS_DIR
# from uqdd.utils import get_config
from uqdd.utils import load_df, create_logger
from uqdd.models.utils_models import load_model, get_model_config
from uqdd.models.baseline import BaselineDNN
from uqdd.models.ensemble import EnsembleDNN
from uqdd.models.mcdropout import mc_predict
from uqdd.models.evidential import EvidentialDNN, ev_predict
from uqdd.models.mcdropout import run_mcdropout # we will use baseline with run_mcdp

# METRICS
from uqdd.models.utils_train import predict, evaluate_predictions, recalibrate_model, get_dataloader

from uqdd.models.utils_metrics import MetricsTable

# # Importing models and their predict functions
# from uqdd.models.utils_train import predict # for ensemble


# setting variables

In [17]:
preds_dirpath = '/home/bkhalil/Repos/uqdd/uqdd/data/predictions/papyrus/xc50/all/'
models_dir = '/users/home/bkhalil/Repos/uqdd/uqdd/models/saved_models/papyrus/xc50/all/'
runs_path = '/users/home/bkhalil/Repos/uqdd/uqdd/figures/papyrus/xc50/all/2024-06-25-all-models-100/runs.csv'
figs_out_path = '/users/home/bkhalil/Repos/uqdd/uqdd/figures/papyrus/xc50/all/reassess-2024-06-25-all-models-100/'
# create figs_out_path if it does not exist
os.makedirs(figs_out_path, exist_ok=True)

In [18]:
# Load csv runs and model names file
runs_df = load_df(runs_path)
runs_df.shape

(80, 14)

In [19]:
runs_df.columns

Index(['Name', 'State', 'Hostname', 'Created', 'data_name',
       'data_specific_path', 'descriptor_chemical', 'descriptor_protein',
       'split_type', 'model_type', 'model_name', 'dropout', 'ckpt_path',
       'ckpt_name'],
      dtype='object')

In [20]:
runs_df.head(-5)

,Name,State,Hostname,Created,data_name,data_specific_path,descriptor_chemical,descriptor_protein,split_type,model_type,model_name,dropout,ckpt_path,ckpt_name
0,devoted-sponge-82,running,wrhel85x1a02525,2024-07-02T15:12:01.000Z,papyrus,NaN,ecfp2048,ankh-large,random,ensemble,NaN,0.1,NaN,NaN
1,leafy-wood-84,finished,wrhel01x5834061,2024-06-27T07:52:49.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,NaN,0.2,NaN,NaN
2,cosmic-field-83,finished,wrhel01x5834061,2024-06-26T09:29:17.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,NaN,0.2,NaN,NaN
3,fast-elevator-82,finished,wrhel01x5834061,2024-06-26T09:28:56.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,NaN,0.2,NaN,NaN
4,noble-frost-81,finished,wrhel01x5834061,2024-06-26T09:28:53.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,NaN,0.2,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,colorful-darkness-9,finished,wrhel01x5834056,2024-06-25T14:11:29.000Z,papyrus,NaN,ecfp2048,ankh-large,random,ensemble,NaN,0.1,NaN,NaN
71,copper-bee-9,finished,wrhel01x5834056,2024-06-25T14:11:29.000Z,papyrus,NaN,ecfp2048,ankh-large,random,ensemble,NaN,0.1,NaN,NaN
72,avid-energy-14,finished,wrhel01x5834056,2024-06-25T14:11:29.000Z,papyrus,NaN,ecfp2048,ankh-large,random,ensemble,NaN,0.1,NaN,NaN
73,peach-wave-9,finished,wrhel01x5834056,2024-06-25T14:11:29.000Z,papyrus,NaN,ecfp2048,ankh-large,random,ensemble,NaN,0.1,NaN,NaN


In [21]:
### Adding model_name
# date-{data_name}_{activity_type}_{split_type}_{descriptor_protein}_{descriptor_chemical}_{data_specific_path}_{run_name}
# 20240625-papyrus_xc50_mcdropout_random_ankh-large_ecfp2048_stoic-firebrand-80
# Fill missing model_name with the required format
runs_df['model_name'] = runs_df.apply(
    lambda row: f"papyrus_xc50_{row['model_type']}_{row['split_type']}_ankh-large_ecfp2048_{row['Name']}"
    if pd.isna(row['model_name']) else row['model_name'], axis=1)

In [22]:
runs_df.head()

,Name,State,Hostname,Created,data_name,data_specific_path,descriptor_chemical,descriptor_protein,split_type,model_type,model_name,dropout,ckpt_path,ckpt_name
0,devoted-sponge-82,running,wrhel85x1a02525,2024-07-02T15:12:01.000Z,papyrus,NaN,ecfp2048,ankh-large,random,ensemble,papyrus_xc50_ensemble_random_ankh-large_ecfp20...,0.1,NaN,NaN
1,leafy-wood-84,finished,wrhel01x5834061,2024-06-27T07:52:49.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,papyrus_xc50_ensemble_scaffold_cluster_ankh-la...,0.2,NaN,NaN
2,cosmic-field-83,finished,wrhel01x5834061,2024-06-26T09:29:17.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,papyrus_xc50_ensemble_scaffold_cluster_ankh-la...,0.2,NaN,NaN
3,fast-elevator-82,finished,wrhel01x5834061,2024-06-26T09:28:56.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,papyrus_xc50_ensemble_scaffold_cluster_ankh-la...,0.2,NaN,NaN
4,noble-frost-81,finished,wrhel01x5834061,2024-06-26T09:28:53.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,papyrus_xc50_ensemble_scaffold_cluster_ankh-la...,0.2,NaN,NaN


In [26]:
# Update model_name
for index, row in runs_df.iterrows():
    model_name = row['model_name']
    model_file_pattern = os.path.join(models_dir, f"*{model_name}.pt")
    model_files = glob.glob(model_file_pattern)
    if model_files:
        model_file_path = model_files[0]
        model_name = os.path.basename(model_file_path).replace('.pt', '')
        runs_df.at[index, 'model_name'] = model_name
        # add model_file_path to the runs_df
        runs_df.at[index, 'model_file_path'] = model_file_path

In [27]:
runs_df.head()

,Name,State,Hostname,Created,data_name,data_specific_path,descriptor_chemical,descriptor_protein,split_type,model_type,model_name,dropout,ckpt_path,ckpt_name,model_file_path
0,devoted-sponge-82,running,wrhel85x1a02525,2024-07-02T15:12:01.000Z,papyrus,NaN,ecfp2048,ankh-large,random,ensemble,papyrus_xc50_ensemble_random_ankh-large_ecfp20...,0.1,NaN,NaN,nan
1,leafy-wood-84,finished,wrhel01x5834061,2024-06-27T07:52:49.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,20240627-papyrus_xc50_ensemble_scaffold_cluste...,0.2,NaN,NaN,/users/home/bkhalil/Repos/uqdd/uqdd/models/sav...
2,cosmic-field-83,finished,wrhel01x5834061,2024-06-26T09:29:17.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,20240626-papyrus_xc50_ensemble_scaffold_cluste...,0.2,NaN,NaN,/users/home/bkhalil/Repos/uqdd/uqdd/models/sav...
3,fast-elevator-82,finished,wrhel01x5834061,2024-06-26T09:28:56.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,20240626-papyrus_xc50_ensemble_scaffold_cluste...,0.2,NaN,NaN,/users/home/bkhalil/Repos/uqdd/uqdd/models/sav...
4,noble-frost-81,finished,wrhel01x5834061,2024-06-26T09:28:53.000Z,papyrus,NaN,ecfp2048,ankh-large,scaffold_cluster,ensemble,20240626-papyrus_xc50_ensemble_scaffold_cluste...,0.2,NaN,NaN,/users/home/bkhalil/Repos/uqdd/uqdd/models/sav...


In [ ]:
def get_model_class(model_type: str):
    if model_type.lower() in ['baseline', 'mcdropout']:
        model_class = BaselineDNN
    elif model_type.lower() == 'ensemble':
        model_class = EnsembleDNN
    elif model_type.lower() == 'evidential':
        model_class = EvidentialDNN
    else:
        raise ValueError(f"Model type {model_type} not recognized")
    return model_class


In [ ]:
logger = create_logger(
    name="reassess",
    file_level="info",
    
)

In [23]:
# Iterate over each run in the runs.csv file
i = 0
for index, row in runs_df.iterrows():
    run_name = row['Name']
    model_type = row['model_type']
    split_type = row['split_type']
    model_name = row['model_name']
    model_path = row['model_file_path']
    if model_path:
        model_class = get_model_class(model_type)
        model = load_model(model_class, model_path)
        config = get_model_config(
            model_type=model_type,
            data_name="papyrus",
            activity_type="xc50",
            split_type=split_type,
            descriptor_protein="ankh-large",
            descriptor_chemical="ecfp2048",
            data_specific_path="papyrus/xc50/all",
            model_name=model_name,
        )
        # Get dataloaders
        dataloaders = get_dataloader(config, device=DEVICE, logger=LOGGER)
        
        
#     if not model_files:
#         print(model_file_pattern)
#         print(model_files)
#         i+=1
# print(i)
    # if index == 5:
    #     break

/users/home/bkhalil/Repos/uqdd/uqdd/models/saved_models/papyrus/xc50/all/*papyrus_xc50_ensemble_random_ankh-large_ecfp2048_devoted-sponge-82.pt
[]
/users/home/bkhalil/Repos/uqdd/uqdd/models/saved_models/papyrus/xc50/all/*papyrus_xc50_ensemble_random_ankh-large_ecfp2048_light-sunset-9.pt
[]
2


In [4]:


# Load predictions for each model_name
# predictions_path = 'predictions.csv'

filename = 'preds.csv'
predictions_df = load_df(preds_dirpath+filename)


FileNotFoundError: [Errno 2] No such file or directory: '/home/bkhalil/Repos/uqdd/uqdd/data/predictions/papyrus/xc50/all/preds.csv'

## Apply some fixes to runs.csv file
1. replace _ between scaffold_cluster with -
2. create the relevant model_name if not present (fixing a bug with ensemble)
3. match with file names after the date to avoid descripancies

In [ ]:
# Iterate over each run in the runs.csv file
for index, row in runs_df.iterrows():
    run_name = row['Name']
    model_type = row['model_type']
    split_type = row['split_type']
    model_name = row['model_name']
    
    model_file_path = os.path.join(models_dir, f"{model_name}.pt")
    
    if os.path.exists(model_file_path):
        
        
        model = load_model(model_class, model_file_path)
        
        # Create the model configuration
        config = get_model_config(
            model_type=model_type,
            data_name="papyrus",
            activity_type="xc50",
            split_type=split_type,
            descriptor_protein="ankh-large",
            descriptor_chemical="ecfp2048",
            data_specific_path="papyrus/xc50/all"
        )
        
        # Get dataloaders
        dataloaders = get_dataloader(config, device=DEVICE, logger=LOGGER)
        
        # Generate predictions
        preds, labels, alea_vars = predict(model, dataloaders["test"], device=DEVICE)
        
        # Calculate the metrics
        metrics, plots, uct_logger = evaluate_predictions(config, preds, labels, alea_vars, model_type, LOGGER)
        
        # Recalibrate model
        preds_val, labels_val, alea_vars_val = predict(model, dataloaders["val"], device=DEVICE)
        iso_recal_model, std_recal = recalibrate_model(preds_val, labels_val, preds, labels, config, uct_logger=uct_logger)
        
        # Log the metrics to the CSV file
        uct_logger.csv_log()

In [ ]:
dataloaders = get_dataloader(config, device=DEVICE, logger=LOGGER)

preds, labels, alea_vars = predict(
    ensemble_model, dataloaders["test"], device=DEVICE
)
# Then comes the predict metrics part
metrics, plots, uct_logger = evaluate_predictions(
    config, preds, labels, alea_vars, "mcdropout", LOGGER
)

# RECALIBRATION # Get Calibration / Validation Set # This predict is changeable according to function
preds_val, labels_val, alea_vars_val = predict(
    ensemble_model, dataloaders["val"], device=DEVICE
)
iso_recal_model, std_recal = recalibrate_model(
    preds_val, labels_val, preds, labels, config_, uct_logger=uct_logger
)



In [1]:

# get arrays from df cols
y_true = predictions_df['y_true'].values
y_pred = predictions_df['y_pred'].values
y_err = predictions_df['y_err'].values
y_alea = predictions_df['y_alea'].values
eps_col = 'y_std' if 'y_std' in predictions_df.columns else 'y_eps'
y_eps = predictions_df[eps_col].values

# we need to get model_type and config from runs_df
model_type = "ensemble"

config = get_model_config(
    model_type,
    data_name="papyrus",
    activity_type="xc50",
    split_type="random",
    descriptor_protein="ankh-large",
    descriptor_chemical="ecfp2048",
    data_specific_path="papyrus/xc50/all"
)


uct_metrics_logger = MetricsTable(
    model_type=model_type,
    config=config,
    add_plots_to_table=False,
    # * we can turn on if we want to see them in wandb * #
    # logger=logger,
    project_name=None, # to change as this will become the output csv file name
    run_name=None, # that would be the same run name from the runs.csv file
)
task_name = "PCM"
metrics, plots = uct_metrics_logger(
        y_pred=y_pred,
        y_std=y_alea,
        y_true=y_true,
        y_err=y_err,
        # y_alea=y_alea,
        y_eps=y_eps,
        task_name=task_name,
        figpath=None, # Here we need to define the fig path 
        # the figpath default was FIGS_DIR / data_specific_path / self.model_name
        # we can change it to mark the reassessment of the model somehoe (maybe add a prefix to model_name like reasses)
        # FIGS_DIR is imported variable from uqdd
        # data_specific_path is predefined var data_specific_path="papyrus/xc50/all"
    )

# * calculate metrics for a subset of the datapoints * #
submetrics, subplots = uct_metrics_logger(
    y_pred=y_pred,
    y_std=y_alea,
    y_true=y_true,
    y_err=y_err,
    # y_alea=y_alea,
    y_eps=y_eps,
    task_name=task_name+" Subset",
    n_subset=100,
)

# then we log to the csv file which appends after each iteration
uct_metrics_logger.csv_log()


NameError: name 'predictions_df' is not defined